<a href="https://colab.research.google.com/github/ayuadmy/simpasi/blob/main/SIMPASI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Needed


In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q scann

In [ ]:
import pathlib
import os
import pprint
import tempfile

from typing import Dict, Text

import csv
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# MPASI Menu, Recipe, Ingredients Dataset

In [ ]:
#Download dataset from google drive
#Bahan_Makanan
!gdown --id 1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS\

#Resep
!gdown --id 1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT\

#Komposisi
!gdown --id 1xufQ7DEIiKIcWWd0sOYXMLkjJQ8kYgvJ\

#Menu Makanan
!gdown --id 1Elz56RH18uKtrV3DDePpT9SKOzhFGVPb\

#Feedback
!gdown --id 1yqDbWgDd8Z9EJawFb0Suy7h3wEDWFR2Y\

#Feedback Bahan
!gdown --id 1GevXM5jVP6jD0wtWPi9LOIxSs3EzHLrY\

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1eenpLlfkok_acgmuIC2XI4tZCnMIkUrS
To: /content/bahan_makanan.csv
100% 5.64k/5.64k [00:00<00:00, 5.20MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1miF99iHy0E0OUfkqP6dZVUccAmW_KpVT
To: /content/resep.csv
100% 3.57k/3.57k [00:00<00:00, 4.43MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From:

In [ ]:
#Move files to tmp (To be deleted after closing this google colab in some time)
!mv "/content/bahan_makanan.csv" "/tmp"
!mv "/content/resep.csv" "/tmp"
!mv "/content/komposisi_baru.csv" "/tmp"
!mv "/content/kombinasi_menu.csv" "/tmp"
!mv "/content/Feedback.csv" "/tmp"
!mv "/content/Feedback_Bahan.csv" "/tmp"

In [ ]:
#Create directory of datasets
bahan_makanan_dir = "/tmp/bahan_makanan.csv"
resep_dir = "/tmp/resep.csv"
komposisi_dir = "/tmp/komposisi_baru.csv"
kombinasi_menu_dir = "/tmp/kombinasi_menu.csv"
feedback_dir = "/tmp/Feedback.csv"
feedback_bahan_dir = "/tmp/Feedback_Bahan.csv"

In [ ]:
bahan_makanan = pd.read_csv(bahan_makanan_dir)
bahan_makanan.head()

,id_bahan,nama_bahan
0,0,Agar-Agar
1,1,Agar Laut
2,2,Akar Tonjong
3,3,Alpukat
4,4,Ampas Tahu


In [ ]:
dict_komposisi = {}

with open(komposisi_dir, mode="r") as data_komposisi:
  reader = csv.reader(data_komposisi)
  keys = next(reader)
  for row in reader:
    if row[0] not in dict_komposisi.keys():
      dict_komposisi[row[0]] = row[2]
    else:
      dict_komposisi[row[0]] = dict_komposisi[row[0]] + ", " + row[2]
    
data = {keys[0]: dict_komposisi.keys(), keys[2]: dict_komposisi.values()}
komposisi = pd.DataFrame.from_dict(data)
komposisi.head()

,id_resep,nama_bahan
0,0,"Blewah, Melon, Agar-Agar"
1,1,Apel
2,2,"Apel, Margarine, Oatmeal"
3,3,"Tahu Sutra, Pisang, Apel"
4,4,"Hati Ayam, Wortel, Kaldu Ayam, Beras, Ayam, Da..."


In [ ]:
resep = pd.read_csv(resep_dir)
resep.head()

,id_resep,nama_resep
0,0,Agar-agar Isi Blewah
1,1,Apel Rebus
2,2,Apple Crumble
3,3,Apricot Tahu
4,4,Ayam Zukini Hati


In [ ]:
komposisi['id_resep'] = komposisi['id_resep'].astype(int)
komposisi_resep = komposisi.merge(resep, on=['id_resep', 'id_resep'], how='left')
#komposisi_resep

del(komposisi_resep['id_resep'])
komposisi_resep = komposisi_resep[['nama_resep', 'nama_bahan']]
komposisi_resep

,nama_resep,nama_bahan
0,Agar-agar Isi Blewah,"Blewah, Melon, Agar-Agar"
1,Apel Rebus,Apel
2,Apple Crumble,"Apel, Margarine, Oatmeal"
3,Apricot Tahu,"Tahu Sutra, Pisang, Apel"
4,Ayam Zukini Hati,"Hati Ayam, Wortel, Kaldu Ayam, Beras, Ayam, Da..."
...,...,...
155,Tim Saring,"Beras, Kaldu Ayam, Ikan Salmon, Beras Merah, W..."
156,Tumis Ikan Dori,"Bawang Merah, Wortel, Ketimun, Bawang Putih, T..."
157,Tuna Mornay,"Saus Pasta, Ikan Tuna, Roti Tawar, Keju, Krim ..."
158,Yoghurt Buah Pop,"Strawbery, Yoghurt"


In [ ]:
menu_harian = pd.read_csv(kombinasi_menu_dir)

menu_harian["malam"].fillna("", inplace = True) 
menu_harian["snack1"].fillna("", inplace = True) 
menu_harian["snack2"].fillna("", inplace = True) 

menu_harian.head()

,id_kombinasi,pagi,siang,malam,snack1,snack2,bulan_6,bulan_7_8,bulan_9_11,bulan_12_24,total_energi,total_carbo,total_protein,total_lipid
0,1,Apel Rebus,Apel Rebus,,,,1,0,0,0,144.00,38.12,0.72,0.46
1,2,Bubur Apel,Bubur Apel,,,,1,0,0,0,144.50,34.52,0.66,0.42
2,3,Bubur Kacang Polong,Bubur Kacang Polong,,,,1,0,0,0,127.44,15.60,3.84,5.52
3,4,Bubur Kembang Kol,Bubur Kembang Kol,,,,1,0,0,0,143.20,21.12,9.46,2.32
4,5,Bubur Plum,Bubur Plum,,,,1,0,0,0,128.10,30.88,1.10,0.02


In [ ]:
list_resep = [a for a in komposisi_resep["nama_resep"]]

cek_resep = menu_harian.drop(["id_kombinasi","bulan_6","bulan_7_8","bulan_9_11","bulan_12_24","total_energi","total_carbo","total_protein","total_lipid"], axis = 1)
komposisi_resep_filtered = komposisi_resep.copy()

for a in list_resep:
  mask = cek_resep.pagi.isin([a])
  if mask.any():
    print("ada bahan pagi {}".format(a))
  else:
    mask = cek_resep.siang.isin([a])
    if mask.any():
      print("ada bahan siang {}".format(a))
    else:
      mask = cek_resep.siang.isin([a])
      if mask.any():
        print("ada bahan malam {}".format(a))
      else:
        mask = cek_resep.snack1.isin([a])
        if mask.any():
          print("ada bahan snack1 {}".format(a))
        else:
          mask = cek_resep.snack2.isin([a])
          if mask.any():
            print("ada bahan snack2 {}".format(a))
          else:
            print("tidak ada bahan {}".format(a))
            komposisi_resep_filtered.drop(komposisi_resep_filtered.index[komposisi_resep_filtered["nama_resep"] == a], inplace=True)

komposisi_resep_filtered

ada bahan snack1 Agar-agar Isi Blewah
ada bahan pagi Apel Rebus
tidak ada bahan Apple Crumble
ada bahan siang Apricot Tahu
tidak ada bahan Ayam Zukini Hati
tidak ada bahan Bayam Panggang
tidak ada bahan Biskuit Beras Merah
ada bahan snack1 Biskuit Oat Ceri
ada bahan snack1 Biskuit Oat Wortel
ada bahan snack1 Biskuit Sereal
ada bahan snack1 Bola-bola Ubi
tidak ada bahan Bola Ikan
tidak ada bahan Brokoli Keju
tidak ada bahan Brokoli Rebus
tidak ada bahan Buah Kebab Dingin
tidak ada bahan Bubur Alpukat
ada bahan pagi Bubur Apel
tidak ada bahan Bubur Apel Bayam Hijau
tidak ada bahan Bubur Apel Bayam Merah
tidak ada bahan Bubur Apel Pisang
tidak ada bahan Bubur Apricot
tidak ada bahan Bubur Aprikot Yogurt
ada bahan pagi Bubur Ayam Kedelai
ada bahan siang Bubur Ayam Talas Keju
tidak ada bahan Bubur Beras
tidak ada bahan Bubur Beras Brocang
ada bahan siang Bubur Beras Merah
tidak ada bahan Bubur Beras Poldu
tidak ada bahan Bubur Bit Ceri
ada bahan pagi Bubur Buah dan Sayur
ada bahan pagi Bubu

,nama_resep,nama_bahan
0,Agar-agar Isi Blewah,"Blewah, Melon, Agar-Agar"
1,Apel Rebus,Apel
3,Apricot Tahu,"Tahu Sutra, Pisang, Apel"
7,Biskuit Oat Ceri,"Unsalted Butter, Oatmeal, Ceri, Maizena"
8,Biskuit Oat Wortel,"Maizena, Unsalted Butter, Wortel, Oatmeal"
...,...,...
147,Sup Labu Kuning,Labu Kuning
149,Sup Mihun Ikan Gabus,"Bihun, Gabus, Kaldu Ikan, Bawang Putih, Margar..."
154,Tim Lele,"Tahu Sutra, Wortel, Belut, Beras"
155,Tim Saring,"Beras, Kaldu Ayam, Ikan Salmon, Beras Merah, W..."


In [ ]:
def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = list(list_set)
    unique_list.sort()

    return unique_list

In [ ]:
resep_bulan_6 = []
resep_bulan_7_8 = []
resep_bulan_9_11 = []
resep_bulan_12_24 = []

cek_bulan = menu_harian.drop(["id_kombinasi","total_energi","total_carbo","total_protein","total_lipid"], axis = 1)

#bulan 6
mask = menu_harian.bulan_6.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_6.append(cek_bulan.pagi[x])
    resep_bulan_6.append(cek_bulan.siang[x])

resep_bulan_6 = unique(resep_bulan_6)
print(resep_bulan_6)

#bulan 7 8
mask = menu_harian.bulan_7_8.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_7_8.append(cek_bulan.pagi[x])
    resep_bulan_7_8.append(cek_bulan.siang[x])
    resep_bulan_7_8.append(cek_bulan.malam[x])

resep_bulan_7_8 = unique(resep_bulan_7_8)
print(resep_bulan_7_8)

#bulan 9 11
mask = menu_harian.bulan_9_11.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_9_11.append(cek_bulan.pagi[x])
    resep_bulan_9_11.append(cek_bulan.siang[x])
    resep_bulan_7_8.append(cek_bulan.malam[x])
    resep_bulan_7_8.append(cek_bulan.snack1[x])

resep_bulan_9_11 = unique(resep_bulan_9_11)
print(resep_bulan_9_11)

#bulan 12 24
mask = menu_harian.bulan_12_24.isin([1])
for x in range(len(mask)):
  if mask[x] == True:
    resep_bulan_12_24.append(cek_bulan.pagi[x])
    resep_bulan_12_24.append(cek_bulan.siang[x])
    resep_bulan_12_24.append(cek_bulan.malam[x])
    resep_bulan_12_24.append(cek_bulan.snack1[x])
    resep_bulan_12_24.append(cek_bulan.snack2[x])

resep_bulan_12_24 = unique(resep_bulan_12_24)
print(resep_bulan_12_24)


['Apel Rebus', 'Apel Rebus', 'Bubur Apel', 'Bubur Apel', 'Bubur Kacang Polong', 'Bubur Kacang Polong', 'Bubur Kembang Kol', 'Bubur Kembang Kol', 'Bubur Plum', 'Bubur Plum', 'Bubur Singkong Apel', 'Bubur Singkong Apel', 'Bubur Susu Kacang Polong', 'Bubur Susu Kacang Polong', 'Bubur Ubi Kembang Kol', 'Bubur Ubi Kembang Kol', 'Pure Alpukat dan Pepaya', 'Pure Alpukat dan Pepaya', 'Pure Apel dan Papaya', 'Pure Apel dan Papaya', 'Pure Jagung Manis', 'Pure Jagung Manis', 'Pure Pisang dan Pepaya', 'Pure Pisang dan Pepaya', 'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning', 'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning', 'Pure Tiga Rasa', 'Pure Tiga Rasa', 'Pure Ubi Kuning', 'Pure Ubi Kuning', 'Pure Ubi Kuning dan Beras Merah', 'Pure Ubi Kuning dan Beras Merah']
['Apel Rebus', 'Bubur Apel', 'Bubur Kacang Polong', 'Bubur Kembang Kol', 'Bubur Plum', 'Bubur Singkong Apel', 'Bubur Susu Kacang Polong', 'Bubur Ubi Kembang Kol', 'Pure Alpukat dan Pepaya', 'Pure Apel dan Papaya', 'Pure Jagung Manis'

# Simulasi Aplikasi SIMPASI - User Baru

Pada user baru, customer akan melewati tahap-tahap berikut akan berupa:

1.   Log-in/Sign-up dengan email dan passsword
2.   Meminta data bayi berupa nama, tanggal lahir, bahan yang pernah dicoba sebelumnya (suka/tidak suka/alergi)
3.   Mendapatkan rekomendasi makanan dari riwayat makanan yang disukai/tidak disukai/alergi




Sehingga, sistem rekomendasi harus melakukan:


1.   Meminta username/email/password untuk masuk ke akun
2.   Meminta nama, tanggal lahir, bahan yang pernah dicoba sebelumnya *(suka/tidak suka/alergi)
3.   Menawarkan resep berdasarkan data bahan
4.   Menghitung Umur dan EER anak
5.   Merekomendasikan resep yang sesuai dengan profil
6.   Memfilter resep berdasarkan alergi bahan
7.   Memfilter menu makanan berdasarkan umur dan EER
8.   Menampilkan kombinasi menu dari resep yang cocok



## I. *Contoh Profil User*

In [ ]:
#Contoh
username = "brianjang"
password = "d8d807375d408c888d0ff9974a82edf4971ed91c0810ec4daa5e47b91bf0de76"
email = "ujang@mail.com"
nama_user = "Ujang Ramabrian"
nama_bayi = "Biyan"
tanggal_lahir = "2021/07/03"
id = "101"
berat_badan = 6.9

## II. Cek 8 Bahan (Suka, Tidak Suka, Alergi, Tidak tahu) dan Feedback Awal 3 Resep


In [ ]:
#Menampilkan Nama Bahan Berdasarkan ID
def ingredient_name(id):
  return bahan_makanan.loc[id, 'nama_bahan']

#driver code
print(ingredient_name(1))

Agar Laut


In [ ]:
#Bahan Alergi Umum: Susu, Telur Ayam, Telur Bebek, Kacang Tanah, Kacang Kedelai, Udang, Kerang, Cumi-Cumi, Tepung Beras, Roti, Daging Sapi, Pisang, Alpukat
tes_bahan = ["Susu", "Telur Ayam", "Telur Bebek", "Kacang Tanah", "Kacang Kedelai", "Udang", "Kerang", "Cumi-Cumi", "Tepung Beras", "Roti Tawar", "Daging Sapi", "Pisang", "Alpukat"]
jumlah_tes = 8
alergi = []
tidak_suka = []
suka = []
tidak_tahu = []

import random
tes_bahan_random = random.sample(tes_bahan, jumlah_tes)

#Meminta 8 Feedback Bahan dari User
print("Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan")
print("Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu")
for i in range(len(tes_bahan_random)):

  x = int(input(tes_bahan_random[i] + ": "))

  if x == 2:
    tidak_suka.append(tes_bahan_random[i])
  elif x == 3:
    alergi.append(tes_bahan_random[i])
  elif x == 4:
    tidak_tahu.append(tes_bahan_random[i])
  elif x == 1:
    suka.append(tes_bahan_random[i])

print(tidak_suka)
print(suka)
print(tidak_tahu)
print(alergi)

Pilih Suka/Tidak Suka/Alergi/Tidak Tahu Pada Bahan Yang Ditampilkan
Ketik 1 = Suka, 2 = Tidak Suka, 3 = Alergi, 4 = Tidak Tahu
Cumi-Cumi: 1
Kacang Tanah: 2
Kacang Kedelai: 4
Susu: 4
Telur Ayam: 1
Kerang: 1
Telur Bebek: 1
Tepung Beras: 3
['Kacang Tanah']
['Cumi-Cumi', 'Telur Ayam', 'Kerang', 'Telur Bebek']
['Kacang Kedelai', 'Susu']
['Tepung Beras']


## III. *Hitung Umur dan EER*

In [ ]:
from datetime import datetime
from datetime import date

#Fungsi Menghitung Umur User
def ageCalculator(birthdate):
  my_birthdate = datetime.strptime(birthdate, "%Y/%m/%d")
  my_year = my_birthdate.year
  my_month = my_birthdate.month
  my_day = my_birthdate.year
  
  born = my_birthdate.date()
  today = date.today()
  
  years = ((today-born).total_seconds()/ (365.242*24*3600))
  yearsInt = int(years)
  months = (years-yearsInt)*12
  monthsInt = int(months)
  days = (months-monthsInt)*(365.242/12)
  daysInt = int(days)

  return yearsInt, monthsInt, daysInt
  

#Fungsi Menghitung EER (Estimated Energy Requirements)
def eerCalculator(year, month, day, weight):
  if month == 6:
    EER = (89*weight - 100) + 56
  elif 7 <= month <= 12:
    EER = (89*weight - 100) + 22
  elif 13 <= month <= 35:
    EER = (89*weight - 100) + 20
  
  return EER
  
#Driver Code
#Cari Umur User (Tahun, Bulan, Tanggal)
tahun, bulan, hari = ageCalculator(tanggal_lahir)
print("Sekarang kamu berumur {0} tahun, {1} bulan, {2} hari.".format(tahun , bulan, hari))

#Cari EER User (+-10%)
eer = eerCalculator(tahun, bulan, hari, berat_badan)
print("Kebutuhan energi harianmu: {0} - {1} Joule".format(int(eer*0.9), int(eer*1.1)))

Sekarang kamu berumur 0 tahun, 10 bulan, 28 hari.
Kebutuhan energi harianmu: 482 - 589 Joule


## IV. *Model Sistem Rekomendasi Menu MPASI*

In [ ]:
df = pd.read_csv(feedback_dir)
feedback_slices = tf.data.Dataset.from_tensor_slices(dict(df))

In [ ]:
#print(bulan)
#menu_filter_umur = []
menu_filter_umur = menu_harian.loc[((menu_harian['bulan_6'] == 1) & (menu_harian['total_energi'] == 482))]
menu_filter_umur.head()
#def filter_umur(month, eer):
#  eer_min = eer*0.9
#  eer_max = eer*1.1

  #for menu_harian["total_energi"] >= eer_min && menu_harian["total_energi"] <= eer_max:
  #  if month <= 6:
  #    return menu_harian.loc[(menu_harian["bulan_6"] == 1)]
  #  elif month in (7, 8):
  #    return menu_harian.loc[(menu_harian["bulan_7_8"] == 1)]
  #  elif month in (9, 10, 11):
  #    return menu_harian.loc[(menu_harian["bulan_9_11"] == 1)]
  #  else:
  #    return menu_harian.loc[(menu_harian["bulan_12"] == 1)]
  
  #menu_filter_eer = menu_harian.loc[(menu_harian['bulan_6'] == 1) & (menu_harian['total_energi'] > eer_min)  & (menu_harian['total_energi'] > eer_min)]
  #print(df2)

#  if month <= 6:
#    menu_filter_umur = kombinasi_menu.loc[(kombinasi_menu['bulan_6'] == 1) & (kombinasi_menu['total_energi'] > eer_min) & (kombinasi_menu['total_energi'] < eer_max)]
#  elif month in (7, 8):
#    menu_filter_umur = kombinasi_menu.loc[(kombinasi_menu['bulan_7_8'] == 1) & (kombinasi_menu['total_energi'] > eer_min) & (kombinasi_menu['total_energi'] < eer_max)]
#  elif month in (9, 10, 11):
#    menu_filter_umur = kombinasi_menu.loc[(kombinasi_menu['bulan_9_11'] == 1) & (kombinasi_menu['total_energi'] > eer_min) & (kombinasi_menu['total_energi'] < eer_max)]
#  else:
#    menu_filter_umur = kombinasi_menu.loc[(kombinasi_menu['bulan_12'] == 1) & (kombinasi_menu['total_energi'] > eer_min) & (kombinasi_menu['total_energi'] < eer_max)]
  #menu_filter_umur.head()
#  menu_filter_umur
#filter_umur(bulan, eer)

,id_kombinasi,pagi,siang,malam,snack1,snack2,bulan_6,bulan_7_8,bulan_9_11,bulan_12_24,total_energi,total_carbo,total_protein,total_lipid


In [ ]:
feedback_slices.element_spec

{'hari': TensorSpec(shape=(), dtype=tf.string, name=None),
 'id_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'id_resep': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'jumlah_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'keterangan': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nilai_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'user_id': TensorSpec(shape=(), dtype=tf.int64, name=None)}

In [ ]:
komposisi_resep_filtered_slices = tf.data.Dataset.from_tensor_slices(dict(komposisi_resep_filtered))

In [ ]:
komposisi_resep_filtered_slices.element_spec

{'nama_bahan': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
feedback = feedback_slices.map(lambda x: {
    "nama_resep": x["nama_resep"],
    "user_id": tf.strings.as_string(x["user_id"]),
})
menu = komposisi_resep_filtered_slices.map(lambda x: x["nama_resep"])

In [ ]:
tf.random.set_seed(12)
shuffled = feedback.shuffle(100, seed=12, reshuffle_each_iteration=False)

train = shuffled.take(80)
test = shuffled.skip(80).take(20)

In [ ]:
nama_resep = menu.batch(10)
user_ids = feedback.batch(100).map(lambda x: (x["user_id"]))

unique_menu = np.unique(np.concatenate(list(nama_resep)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
nama_resep.element_spec

TensorSpec(shape=(None,), dtype=tf.string, name=None)

In [ ]:
user_ids.element_spec

TensorSpec(shape=(None,), dtype=tf.string, name=None)

In [ ]:
unique_menu[:10]

array([b'Apel Rebus', b'Bubur Apel', b'Bubur Ayam Kedelai',
       b'Bubur Buah dan Sayur', b'Bubur Ceri Alpukat',
       b'Bubur Jagung Bayam Tahu', b'Bubur Kacang Hijau Kurma',
       b'Bubur Kacang Polong', b'Bubur Kembang Kol', b'Bubur Kentang'],
      dtype=object)

In [ ]:
unique_user_ids[:10]

array([b'1', b'11', b'12', b'15', b'19', b'23', b'25', b'26', b'28',
       b'31'], dtype=object)

In [ ]:
embedding_dimension = 8

In [ ]:
#Query Tower
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
#Candidate Tower
menu_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_menu, mask_token=None),
  tf.keras.layers.Embedding(len(unique_menu) + 1, embedding_dimension)
])

In [ ]:
#metrics
metrics = tfrs.metrics.FactorizedTopK(
  candidates=menu.batch(64).map(menu_model)
)

In [ ]:
#loss
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
class SIMPASIModel(tfrs.Model):

  def __init__(self, user_model, menu_model):
    super().__init__()
    self.menu_model: tf.keras.Model = menu_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_menu_embeddings = self.menu_model(features["nama_resep"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_menu_embeddings)

In [ ]:
class NoBaseClassSIMPASIModel(tf.keras.Model):

  def __init__(self, user_model, menu_model):
    super().__init__()
    self.menu_model: tf.keras.Model = menu_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.user_model(features["user_id"])
      positive_menu_embeddings = self.menu_model(features["nama_resep"])
      loss = self.task(user_embeddings, positive_menu_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.user_model(features["user_id"])
    positive_menu_embeddings = self.movie_model(features["nama_resep"])
    loss = self.task(user_embeddings, positive_menu_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [ ]:
model = SIMPASIModel(user_model, menu_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(10).batch(16).cache()
cached_test = test.batch(8).cache()

In [ ]:
model.fit(cached_train, epochs=5)

Epoch 1/5
5/5 [==============================] - 1s 58ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.1000 - factorized_top_k/top_10_categorical_accuracy: 0.2250 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 44.3569 - regularization_loss: 0.0000e+00 - total_loss: 44.3569
Epoch 2/5
5/5 [==============================] - 0s 54ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1250 - factorized_top_k/top_5_categorical_accuracy: 0.3875 - factorized_top_k/top_10_categorical_accuracy: 0.6750 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 44.3018 - regularization_loss: 0.0000e+00 - total_loss: 44.3018
Epoch 3/5
5/5 [==============================] - 0s 58ms/step - factorized_top_k/top_1_categorical_accuracy: 0.3125 - factorized_top_k/top_5_categorical_accuracy: 0.6875 - factoriz

In [ ]:
model.evaluate(cached_test, return_dict=True)

3/3 [==============================] - 1s 55ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2000 - factorized_top_k/top_5_categorical_accuracy: 0.2000 - factorized_top_k/top_10_categorical_accuracy: 0.2500 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 11.7859 - regularization_loss: 0.0000e+00 - total_loss: 11.7859


{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.25,
 'factorized_top_k/top_1_categorical_accuracy': 0.20000000298023224,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.20000000298023224,
 'loss': 5.970164775848389,
 'regularization_loss': 0,
 'total_loss': 5.970164775848389}

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model, 41)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((menu.batch(100), menu.batch(100).map(model.menu_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["10"]))
print(f"Recommendations for user 10: {titles}")

Recommendations for user 10: [[b'Bubur Ceri Alpukat' b'Bubur Jagung Bayam Tahu' b'Pure Jagung Manis'
  b'Bubur Buah dan Sayur' b'Bubur Pisang Tomat' b'Bubur Labu Brokoli'
  b'Bubur Tahu' b'Bubur Ayam Kedelai' b'Bubur Pepaya'
  b'Bubur Susu Kacang Polong' b'Bubur Kacang Hijau Kurma'
  b'Sup Mihun Ikan Gabus' b'Apel Rebus' b'Bubur Pisang Ubi'
  b'Oatmeal Kurma' b'Pure Pisang dan Pepaya' b'Bubur Oatmeal'
  b'Bubur Tempe' b'Bubur Pisang Kentang' b'Pure Ubi Kuning'
  b'Bubur Sawi Wortel' b'Bubur Apel' b'Bubur Pisang' b'Bubur Talas Apel'
  b'Bubur Kentang' b'Bubur Singkong Apel' b'Bubur Wortel'
  b'Pure Tiga Rasa' b'Bubur Plum' b'Bubur Kembang Kol'
  b'Bubur Ubi Bayam Kacang Polong' b'Pisang Mangga Rebus'
  b'Bubur Kacang Polong' b'Sayur Kukus' b'Pure Alpukat dan Pepaya'
  b'Bubur Kentang Edamame' b'Pure Ubi Kuning dan Beras Merah'
  b'Pure Apel dan Papaya' b'Bubur Pir'
  b'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning'
  b'Bubur Ubi Kembang Kol']]


In [ ]:
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["42"])

  print(f"Recommendations: {titles[0][:41]}")

Recommendations: [b'Bubur Pisang Tomat' b'Bubur Talas Apel' b'Bubur Tahu' b'Apel Rebus'
 b'Bubur Sawi Wortel' b'Bubur Singkong Apel'
 b'Bubur Ubi Bayam Kacang Polong' b'Bubur Pepaya' b'Bubur Ceri Alpukat'
 b'Pure Alpukat dan Pepaya' b'Oatmeal Kurma' b'Pure Ubi Kuning'
 b'Bubur Pisang Kentang' b'Bubur Tempe' b'Bubur Kentang'
 b'Sup Mihun Ikan Gabus' b'Bubur Kembang Kol' b'Pure Jagung Manis'
 b'Bubur Apel' b'Bubur Oatmeal' b'Pure Pisang dan Pepaya'
 b'Bubur Kacang Polong' b'Bubur Kacang Hijau Kurma' b'Bubur Ayam Kedelai'
 b'Bubur Susu Kacang Polong' b'Pure Ubi Kuning dan Beras Merah'
 b'Bubur Kentang Edamame' b'Bubur Pisang Ubi' b'Pisang Mangga Rebus'
 b'Pure Apel dan Papaya']


In [ ]:
print(type(titles))            # <type 'EagerTensor'>
print(type(titles.numpy()))    # <type 'numpy.ndarray'>
print(type(np.array(titles)))  # <type 'numpy.ndarray'>

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
titles_numpy = np.array(titles).transpose().astype(str)

rekomendasi = pd.DataFrame(titles_numpy, columns=["rekomendasi_resep"])
rekomendasi.head()

,rekomendasi_resep
0,Bubur Ceri Alpukat
1,Bubur Jagung Bayam Tahu
2,Pure Jagung Manis
3,Bubur Buah dan Sayur
4,Bubur Pisang Tomat


## V. *Filtering Rekomendasi Resep Berdasarkan Alergi*

In [ ]:
def search_list(list1):
  pattern = '|'.join([f'(?i){list_value}' for list_value in list1])
  return pattern

In [ ]:
list_rekomendasi = [a for a in rekomendasi["rekomendasi_resep"]]
print(len(list_rekomendasi))
# mask = komposisi_resep_filtered.nama_bahan.isin(list_rekomendasi)
# for i in mask.index:
#   if mask[x] == True:
#     resep_bulan_6.append(cek_bulan.pagi[x])
#     resep_bulan_6.append(cek_bulan.siang[x])

#   mask = komposisi_resep_filtered.nama_resep.isin([rekomendasi['rekomendasi_resep'][i]])
  
#   for x in range(len(mask)):
#     if mask[x] == True:
#       komposisi_resep_filtered.nama_bahan[x]


mask = komposisi_resep_filtered.nama_resep.isin(list_rekomendasi)
rekomendasi_resep_alergi = komposisi_resep_filtered[mask]
len(rekomendasi_resep_alergi)
rekomendasi_resep_alergi

41


,nama_resep,nama_bahan
1,Apel Rebus,Apel
16,Bubur Apel,Apel
22,Bubur Ayam Kedelai,"Kacang Kedelai, Ayam, Kaldu Ayam"
29,Bubur Buah dan Sayur,"Strawbery, Brokoli"
30,Bubur Ceri Alpukat,"Yoghurt, Alpukat, Ceri"
34,Bubur Jagung Bayam Tahu,"Bayam, Tahu Sutra, Kaldu Ayam, Jagung"
37,Bubur Kacang Hijau Kurma,"Kurma, Kacang Buncis, Kacang Ijo"
38,Bubur Kacang Polong,Kacang Polong
40,Bubur Kembang Kol,Kembang Kol
41,Bubur Kentang,Kentang


In [ ]:
mask = rekomendasi_resep_alergi.nama_bahan.str.contains(search_list(alergi))
rekomendasi_resep_alergi = rekomendasi_resep_alergi[~mask]
len(rekomendasi_resep_alergi)
rekomendasi_resep_alergi

                          nama_resep  \
1                         Apel Rebus   
16                        Bubur Apel   
22                Bubur Ayam Kedelai   
29              Bubur Buah dan Sayur   
30                Bubur Ceri Alpukat   
34           Bubur Jagung Bayam Tahu   
37          Bubur Kacang Hijau Kurma   
38               Bubur Kacang Polong   
40                 Bubur Kembang Kol   
41                     Bubur Kentang   
43             Bubur Kentang Edamame   
46                Bubur Labu Brokoli   
50                     Bubur Oatmeal   
51                      Bubur Pepaya   
53                         Bubur Pir   
54                      Bubur Pisang   
56              Bubur Pisang Kentang   
58                Bubur Pisang Tomat   
59                  Bubur Pisang Ubi   
60                        Bubur Plum   
61                 Bubur Sawi Wortel   
63               Bubur Singkong Apel   
65          Bubur Susu Kacang Polong   
66                        Bubur Tahu   


40

## VI. *Filtering Rekomendasi Resep Berdasarkan umur dan EER*